In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from shapely.geometry import Point, LineString, MultiLineString
import geopandas as gpd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import warnings
warnings.filterwarnings("ignore")


In [ ]:
df_train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/train.csv')

In [ ]:
df_train

In [ ]:
df_train_streets = df_train.drop_duplicates(subset=['x','y','direction']).copy()
def reversed_bool(direction):
    if direction == 'WB':
        return True
    elif direction == 'SB':
        return True
    elif direction == 'SW':
        return True
    elif direction == 'SE':
        return True
    else:
        return False
def unify_directions(direction):
    if direction == 'WB':
        return 'EB'
    elif direction == 'SB':
        return 'NB'
    elif direction == 'SW':
        return 'NE'
    elif direction == 'SE':
        return 'NW'
    else:
        return direction
df_train_streets['reversed'] = df_train_streets.direction.apply(reversed_bool)
df_train_streets['direction'] = df_train_streets.direction.apply(unify_directions)
df_train_streets.drop_duplicates(subset=['x','y','direction','reversed'], inplace=True)
#df_train_streets['geometry'] = df_train.apply(lambda x: Point([x['x'],x['y']]), axis=1)

In [ ]:
df_streets = df_train_streets
df_streets.drop(['row_id','time','congestion'], axis=1, inplace=True)
df_streets

In [ ]:
from sklearn.preprocessing import LabelEncoder

df_streets['xydirection'] = df_streets.apply(lambda row: str(row['x']) + str(row['y']) + row['direction'], axis=1)
le = LabelEncoder()
df_streets['street_id'] = le.fit_transform(df_streets['xydirection'])
df_streets.drop('xydirection', axis=1, inplace=True)
df_streets

In [ ]:
unreversed_streets = df_streets.loc[df_streets.reversed == True].copy()
reversed_streets = df_streets.loc[df_streets.reversed == False].copy()
unreversed_streets.drop('reversed', axis=1, inplace=True)
reversed_streets.drop('reversed', axis=1, inplace=True)

all_streets = reversed_streets.merge(unreversed_streets, how='outer', indicator=True)
one_way_streets = pd.concat([all_streets.loc[all_streets._merge == 'left_only'], all_streets.loc[all_streets._merge == 'right_only']])
two_way_streets = all_streets.loc[all_streets._merge == 'both']
one_way_streets.drop('_merge', axis=1, inplace=True)
two_way_streets.drop('_merge', axis=1, inplace=True)
one_way_streets

In [ ]:
two_way_streets

In [ ]:
one_way_streets['oneway'] = True
two_way_streets['oneway'] = False
all_streets = pd.concat([one_way_streets, two_way_streets]).copy()
one_way_streets.drop('oneway', axis=1, inplace=True)
two_way_streets.drop('oneway', axis=1, inplace=True)

def point_direction_toline(row):
    x = row['x']
    y = row['y']
    direction = row['direction']
    if direction == 'EB':
        return LineString([(x-1,y),(x+1,y)])
    elif direction == 'NB' :
        return LineString([(x,y-1),(x,y+1)])
    elif direction == 'NE':
        return LineString([(x-1,y-1),(x+1,y+1)])
    elif direction == 'NW':
        return LineString([(x-1,y+1),(x+1,y-1)])
all_streets['geometry'] = all_streets.apply(point_direction_toline, axis=1)

all_streets_gdf = gpd.GeoDataFrame(all_streets, geometry='geometry')
all_streets_gdf.plot('oneway', figsize=(12,12))

In [ ]:
import networkx as nx

G = nx.Graph()
G.add_nodes_from(all_streets.street_id)

two_way_streets['geometry'] = two_way_streets.apply(point_direction_toline, axis=1)
two_way_streets_gdf = gpd.GeoDataFrame(two_way_streets, geometry='geometry')
one_way_streets['geometry'] = one_way_streets.apply(point_direction_toline, axis=1)
one_way_streets_gdf = gpd.GeoDataFrame(one_way_streets, geometry='geometry')


In [ ]:
for i, row1 in two_way_streets_gdf.iterrows():
    line1 = row1['geometry']
    street1 = row1['street_id']
    for j, row2 in two_way_streets_gdf.iterrows():
        line2 = row2['geometry']
        street2 = row2['street_id']
        if i != j:
            if line1.touches(line2) or line1.intersects(line2):
                G.add_edge(street1, street2)

In [ ]:
nx.draw(G, width=0.5)